# CAPA 0

In [1]:
# -*- coding: utf-8 -*-
from __future__ import annotations
from dataclasses import dataclass, replace
from typing import Protocol, Literal, Union, Mapping, Optional, Dict, List, Tuple
import math
import numpy as np
import pandas as pd
import logging
import hashlib

# Configuración del sistema de logging para trazabilidad de la simulación
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("capa0_simulation.log"),
        logging.StreamHandler()
    ]
)

# Tipo auxiliar para manejar tanto valores escalares como series temporales
NumberOrSeries = Union[float, int, pd.Series]

# ---------------------------------------------------------------------
# Estructuras de datos para entrada/salida
# ---------------------------------------------------------------------

@dataclass
class PerfilSuelo:
    """
    Estructura del perfil de suelo multicapa (4 capas: 20, 40, 60, 90 cm).
    La estratificación permite capturar la heterogeneidad vertical de los
    procesos hídricos en el viñedo, diferenciando evaporación superficial,
    infiltración progresiva y extracción radicular por profundidad.
    """
    profundidades: List[str]          # Etiquetas de profundidad (cm) Ej: ['20','40','60','90'] 
    theta: Dict[str, NumberOrSeries]  # Contenido volumétrico de agua actual (m³/m³)
    FC: Dict[str, float]              # Capacidad de campo por capa (m³/m³)
    PWP: Dict[str, float]             # Punto de marchitez permanente por capa (m³/m³)
    pesos: Dict[str, float]           # Pesos de contribución radicular por capa
    
    def validar(self) -> bool:
        """Validación de consistencia física del perfil de suelo"""
        comunes = (set(self.theta.keys()) & set(self.FC.keys()) &
                   set(self.PWP.keys()) & set(self.pesos.keys()))
        if not comunes:
            raise ValueError("Las claves de profundidad no coinciden entre theta/FC/PWP/pesos.")
        
        # Validar coherencia física: FC debe estar siempre por encima de PWP
        for d in comunes:
            if self.FC[d] <= self.PWP[d]:
                raise ValueError(f"FC debe ser > PWP en capa {d}.")
        
        # Validar rango físico de theta [0,1] por capa
        for d in comunes:
            th = self.theta[d]
            if isinstance(th, pd.Series):
                if ((th < 0) | (th > 1)).any():
                    raise ValueError(f"theta fuera de [0,1] en capa {d}.")
            else:
                if not (0.0 <= float(th) <= 1.0):
                    raise ValueError(f"theta={th} fuera de [0,1] en capa {d}.")
        
        # Normalizar pesos si no suman exactamente 1.0
        total_pesos = sum(self.pesos[d] for d in comunes)
        if abs(total_pesos - 1.0) > 1e-2:
            for d in comunes:
                self.pesos[d] /= total_pesos
        
        # Verificar que todas las series temporales compartan el mismo índice
        indices = []
        for d in comunes:
            if isinstance(self.theta[d], pd.Series):
                indices.append(self.theta[d].index)
        if indices and not all(idx.equals(indices[0]) for idx in indices):
            raise ValueError("Todas las series de theta deben compartir el mismo índice temporal.")
        
        return True
    
    def copiar(self) -> PerfilSuelo:
        return PerfilSuelo(
            profundidades=self.profundidades.copy(),
            theta=self.theta.copy(),
            FC=self.FC.copy(),
            PWP=self.PWP.copy(),
            pesos=self.pesos.copy()
        )

@dataclass(frozen=True)
class PasoEntrada:
    """
    Datos de entrada para un paso de cálculo. Agrupa variables meteorológicas FAO-56,
    parámetros de cultivo/suelo, y estado del perfil. Frozen=True evita modificaciones
    accidentales y permite hasheo para caching.
    """
    # Variables meteorológicas FAO-56
    Tmin: NumberOrSeries  # Temperatura mínima diaria (°C)
    Tmax: NumberOrSeries  # Temperatura máxima diaria (°C)
    RHmin: NumberOrSeries  # Humedad relativa mínima (%)
    RHmax: NumberOrSeries  # Humedad relativa máxima (%)
    Uz: NumberOrSeries  # Velocidad del viento a altura z (m/s)
    z_viento: float  # Altura de medición del viento (m)
    z_terreno: float  # Altitud sobre el nivel del mar (m)
    lat_deg: float  # Latitud de la parcela (grados decimales)
    J: NumberOrSeries  # Día juliano del año [1-366]
    Rs: NumberOrSeries  # Radiación solar medida (MJ/m²/día)
    
    # Variables de cultivo y aportes hídricos
    h_m: NumberOrSeries  # Altura media del cultivo (m)
    fc: NumberOrSeries  # Fracción de cobertura vegetal [0-1]
    fw: NumberOrSeries  # Fracción de suelo mojado por riego/lluvia [0-1]
    lluvia: NumberOrSeries  # Precipitación bruta del día (mm)
    lluvia_esperada_0_24h: NumberOrSeries  # Previsión de lluvia 0-24h (mm)
    lluvia_esperada_24_48h: NumberOrSeries  # Previsión de lluvia 24-48h (mm)
    irrig: NumberOrSeries  # Riego neto aplicado (mm, eficiencia ya descontada)
    
    # Parámetros Kcb por fase fenológica
    kcb_init: NumberOrSeries  # Coeficiente basal cultivo fase inicial [0-1.5]
    kcb_mid: NumberOrSeries  # Coeficiente basal cultivo fase media [0-1.5]
    kcb_method: Literal["linear", "given"]  # Método de cálculo: interpolación lineal o valor fijo
    
    # Estado de la capa superficial evaporativa
    De: NumberOrSeries  # Agotamiento de agua evaporable en superficie (mm)
    TEW: NumberOrSeries  # Agua total evaporable en superficie (mm)
    REW: NumberOrSeries  # Agua fácilmente evaporable en superficie (mm)
    
    # Perfil de suelo unificado
    perfil_suelo: Optional[PerfilSuelo] = None

@dataclass(frozen=True)
class PasoSalida:
    """
    Resultados del cálculo de la Capa 0. Incluye indicadores principales para las
    capas superiores y variables de trazabilidad para diagnóstico y auditoría.
    """
    # Indicadores principales (consumidos por Capas 1 y 2)
    Ks: NumberOrSeries  # Coeficiente de estrés hídrico [0-1]
    J: NumberOrSeries  # Día juliano [1-366]
    pe_prevista_0_24h: NumberOrSeries  # Precipitación efectiva prevista 0-24h (mm)
    pe_prevista_24_48h: NumberOrSeries  # Precipitación efectiva prevista 24-48h (mm)
    Pe: NumberOrSeries  # Precipitación efectiva actual (mm)
    ETc: NumberOrSeries  # Evapotranspiración del cultivo (mm/día)
    RSWC: NumberOrSeries  # Contenido relativo de agua en el suelo [0-1]
    
    # Variables de trazabilidad y diagnóstico
    DND: Optional[NumberOrSeries] = None  # Demanda neta diaria ETc-Pe (mm/día)
    TAW: Optional[float] = None  # Agua total disponible en el perfil (mm)
    RAW: Optional[NumberOrSeries] = None  # Agua fácilmente utilizable (mm)
    Dr: Optional[NumberOrSeries] = None  # Déficit de agua en el perfil (mm)
    p: Optional[NumberOrSeries] = None  # Fracción de agotamiento permitido [0-1]
    Kcb: Optional[NumberOrSeries] = None  # Coeficiente basal de cultivo [0-1.5]
    Ke: Optional[NumberOrSeries] = None  # Coeficiente de evaporación del suelo [0-1.5]
    Kc: Optional[NumberOrSeries] = None  # Coeficiente de cultivo total [0-1.5]
    Kcmax: Optional[NumberOrSeries] = None  # Coeficiente de cultivo máximo tras mojado [0-1.5]
    Kr: Optional[NumberOrSeries] = None  # Coeficiente de reducción de evaporación [0-1]
    few: Optional[NumberOrSeries] = None  # Fracción de suelo húmedo expuesto [0-1]
    De_end: Optional[NumberOrSeries] = None  # Agotamiento superficial al final del día (mm)
    ETo: Optional[NumberOrSeries] = None  # Evapotranspiración de referencia (mm/día)
    
    # Notas de control de calidad
    qc_notes: Optional[List[str]] = None  # Registro de correcciones/warnings aplicados

@dataclass
class SimulationState:
    """Estado de la simulación para encadenamiento entre días"""
    perfil_suelo: PerfilSuelo  # Estado del perfil multicapa
    De: NumberOrSeries  # Agotamiento superficial actual (mm)
    day: int  # Contador de día de simulación
    
    def copiar(self) -> SimulationState:
        return SimulationState(
            perfil_suelo=self.perfil_suelo.copiar(),
            De=self.De,
            day=self.day
        )

# ---------------------------------------------------------------------
# Strategy Pattern para cálculo de ETc
# ---------------------------------------------------------------------

class ETcStrategy(Protocol):
    """Protocolo para estrategias de cálculo de ETc"""
    def __call__(self, x: PasoEntrada, ETo: NumberOrSeries, Ks: NumberOrSeries) -> Dict[str, NumberOrSeries]:
        ...

# ---------------------------------------------------------------------
# Funciones auxiliares
# ---------------------------------------------------------------------

def clip01(x: NumberOrSeries) -> NumberOrSeries:
    """Restringe valores al rango [0,1]"""
    if isinstance(x, pd.Series):
        return x.clip(0, 1)
    return max(0.0, min(1.0, float(x)))

def validar_rango(valor: NumberOrSeries, nombre: str, min_val: float, max_val: float):
    """Valida que un valor esté dentro de rangos físicamente plausibles"""
    if isinstance(valor, pd.Series):
        if (valor < min_val).any() or (valor > max_val).any():
            raise ValueError(f"{nombre} tiene valores fuera del rango [{min_val}, {max_val}]")
    else:
        if not min_val <= float(valor) <= max_val:
            raise ValueError(f"{nombre} = {valor} fuera del rango [{min_val}, {max_val}]")

def calc_Pe(lluvia: NumberOrSeries) -> NumberOrSeries:
    """
    Precipitación efectiva según aproximación USDA-SCS adaptada a clima mediterráneo.
    En estas condiciones, se descarta lluvia < 3 mm y solo una fracción del resto
    resulta utilizable por el cultivo. Pe = max(0, 0.75 * (P - 3))
    """
    if isinstance(lluvia, pd.Series):
        return lluvia.apply(lambda v: max(0.0, 0.75 * (float(v) - 3.0)))
    return max(0.0, 0.75 * (float(lluvia) - 3.0))

def calc_RSWC_weighted(theta_by_depth: Mapping[str, NumberOrSeries],
                       FC_by_depth: Mapping[str, float],
                       PWP_by_depth: Mapping[str, float],
                       weights: Mapping[str, float]) -> NumberOrSeries:
    """
    Contenido relativo de agua en el suelo (RSWC) mediante integración ponderada
    por pesos de contribución radicular. Captura la heterogeneidad vertical del
    estado hídrico mejor que enfoques monolíticos.
    """
    depths = [d for d in weights.keys() if d in theta_by_depth and d in FC_by_depth and d in PWP_by_depth]
    if not depths:
        raise ValueError("No hay capas comunes entre theta/FC/PWP/weights.")
    
    parts = []
    eff_weights = []
    for d in depths:
        FC_d = FC_by_depth[d]
        PWP_d = PWP_by_depth[d]
        denom = FC_d - PWP_d
        if denom <= 0:
            continue
        
        theta_d = theta_by_depth[d]
        if isinstance(theta_d, pd.Series):
            rswc_i = ((theta_d - PWP_d) / denom).clip(0, 1)
        else:
            val = (float(theta_d) - PWP_d) / denom
            rswc_i = max(0.0, min(1.0, val))
        
        parts.append(rswc_i)
        eff_weights.append(weights[d])
    
    wsum = sum(eff_weights)
    if wsum <= 0:
        raise ValueError("La suma de pesos efectivos es 0.")
    eff_weights = [w / wsum for w in eff_weights]
    
    # Alinear series si es necesario
    if any(isinstance(p, pd.Series) for p in parts):
        idx = None
        parts_aligned: List[pd.Series] = []
        for p in parts:
            if isinstance(p, pd.Series):
                idx = p.index if idx is None else idx
                parts_aligned.append(p)
            else:
                parts_aligned.append(pd.Series([p] * len(idx), index=idx))
        return sum(w * p for w, p in zip(eff_weights, parts_aligned))
    
    return float(sum(w * float(p) for w, p in zip(eff_weights, parts)))

# ---------------------------------------------------------------------
# Cálculo de espesores y parámetros del perfil
# ---------------------------------------------------------------------

def thickness_map_from_profundidades(profundidades: List[str]) -> Dict[str, float]:
    """
    Calcula espesor de cada capa a partir de profundidades acumuladas.
    Mantiene las etiquetas originales para evitar desajustes en diccionarios.
    """
    pares = sorted([(float(p), p) for p in profundidades], key=lambda t: t[0])
    tm: Dict[str, float] = {}
    prev = 0.0
    for depth_cm, label in pares:
        tm[label] = depth_cm - prev
        prev = depth_cm
    return tm

def calc_TAW(perfil_suelo: PerfilSuelo) -> float:
    """
    Agua Total Disponible en el perfil (mm). TAW = 10 * Σ[(FC_i - PWP_i) * espesor_i]
    El factor 10 convierte de (m³/m³ * cm) a mm.
    """
    tm = thickness_map_from_profundidades(perfil_suelo.profundidades)
    return sum((perfil_suelo.FC[d] - perfil_suelo.PWP[d]) * tm[d] * 10.0 for d in tm)

def calc_RAW(TAW: float, p: NumberOrSeries) -> NumberOrSeries:
    """
    Agua Fácilmente Utilizable (mm). RAW = p * TAW, donde p es el factor de
    agotamiento permitido antes de que se inicie estrés hídrico.
    """
    if isinstance(p, pd.Series):
        return p * TAW
    return p * TAW

def calc_Dr(perfil_suelo: PerfilSuelo) -> NumberOrSeries:
    """
    Déficit de agua en el perfil (mm). Dr = 10 * Σ[(FC_i - θ_i) * espesor_i]
    Refleja el agotamiento acumulado respecto a capacidad de campo.
    """
    tm = thickness_map_from_profundidades(perfil_suelo.profundidades)
    Dr: Optional[NumberOrSeries] = None
    for d in tm:
        term = (perfil_suelo.FC[d] - perfil_suelo.theta[d]) * tm[d] * 10.0
        if Dr is None:
            Dr = term
        else:
            # Manejar suma de escalares y series
            if isinstance(Dr, pd.Series) or isinstance(term, pd.Series):
                if not isinstance(Dr, pd.Series):
                    Dr = pd.Series([Dr] * len(term.index), index=term.index)
                if not isinstance(term, pd.Series):
                    term = pd.Series([term] * len(Dr.index), index=Dr.index)
            Dr = Dr + term
    return Dr if Dr is not None else 0.0

def calc_Ks(Dr: NumberOrSeries, TAW: float, RAW: NumberOrSeries) -> NumberOrSeries:
    """
    Coeficiente de estrés hídrico (adimensional). Ks=1 cuando Dr≤RAW (sin estrés),
    y disminuye linealmente hasta 0 cuando Dr→TAW. Solo afecta a la transpiración
    del cultivo (Kcb), no a la evaporación del suelo (Ke).
    """
    if isinstance(Dr, pd.Series):
        Ks = pd.Series(1.0, index=Dr.index)
        if isinstance(RAW, pd.Series):
            mask = Dr > RAW
            Ks[mask] = (TAW - Dr[mask]) / (TAW - RAW[mask])
        else:
            mask = Dr > RAW
            Ks[mask] = (TAW - Dr[mask]) / (TAW - RAW)
        return Ks.clip(0, 1)
    else:
        if Dr <= RAW:
            return 1.0
        return max(0.0, min(1.0, (TAW - Dr) / (TAW - RAW)))

def calc_p_adjusted(p_ref: float, ETo: NumberOrSeries) -> NumberOrSeries:
    """
    Factor de agotamiento ajustado dinámicamente por demanda evaporativa.
    p = p_ref + 0.04*(5 - ETo), limitado a [0.1, 0.8]. Con mayor ETo, el cultivo
    es más sensible al déficit (p disminuye), activando el riego antes.
    """
    if isinstance(ETo, pd.Series):
        p = p_ref + 0.04 * (5 - ETo)
        return p.clip(0.1, 0.8)
    p = p_ref + 0.04 * (5 - float(ETo))
    return max(0.1, min(0.8, p))

# ---------------------------------------------------------------------
# Cálculo de ETo según FAO-56 Penman-Monteith
# ---------------------------------------------------------------------

def esat_scalar(T: float) -> float:
    """Presión de vapor de saturación a temperatura T (kPa)"""
    return 0.6108 * math.exp(17.27 * T / (T + 237.3))

def esat_vectorized(T: pd.Series) -> pd.Series:
    return 0.6108 * np.exp(17.27 * T / (T + 237.3))

def es_daily_scalar(Tmin: float, Tmax: float) -> float:
    """Presión de vapor de saturación media diaria (kPa)"""
    return 0.5 * (esat_scalar(Tmin) + esat_scalar(Tmax))

def es_daily_vectorized(Tmin: pd.Series, Tmax: pd.Series) -> pd.Series:
    return 0.5 * (esat_vectorized(Tmin) + esat_vectorized(Tmax))

def ea_from_RH_scalar(Tmin: float, Tmax: float, RHmin: float, RHmax: float) -> float:
    """Presión real de vapor a partir de HR mínima y máxima (kPa)"""
    return 0.5 * (esat_scalar(Tmin) * (RHmax / 100.0) + esat_scalar(Tmax) * (RHmin / 100.0))

def ea_from_RH_vectorized(Tmin: pd.Series, Tmax: pd.Series,
                          RHmin: pd.Series, RHmax: pd.Series) -> pd.Series:
    return 0.5 * (esat_vectorized(Tmin) * (RHmax / 100.0) +
                  esat_vectorized(Tmax) * (RHmin / 100.0))

def delta_slope_scalar(Tmean: float) -> float:
    """Pendiente de la curva de presión de vapor (kPa/°C)"""
    es = esat_scalar(Tmean)
    return 4098.0 * es / ((Tmean + 237.3) ** 2)

def delta_slope_vectorized(Tmean: pd.Series) -> pd.Series:
    es = esat_vectorized(Tmean)
    return 4098.0 * es / ((Tmean + 237.3) ** 2)

def pressure_from_z_scalar(z: float) -> float:
    """Presión atmosférica en función de la altitud (kPa)"""
    return 101.3 * ((293.0 - 0.0065 * z) / 293.0) ** 5.26

def pressure_from_z_vectorized(z: float, size: int) -> pd.Series:
    return pd.Series([pressure_from_z_scalar(z)] * size)

def lambda_from_T_scalar(Tmean: float) -> float:
    """Calor latente de vaporización (MJ/kg)"""
    return 2.501 - 0.002361 * Tmean

def lambda_from_T_vectorized(Tmean: pd.Series) -> pd.Series:
    return 2.501 - 0.002361 * Tmean

def psychrometric_gamma_scalar(P: float, lam: float) -> float:
    """Constante psicrométrica (kPa/°C)"""
    return 0.00163 * (P / lam)

def psychrometric_gamma_vectorized(P: pd.Series, lam: pd.Series) -> pd.Series:
    return 0.00163 * (P / lam)

def Ra_MJm2day_scalar(lat_deg: float, J: int) -> float:
    """Radiación extraterrestre (MJ/m²/día)"""
    Gsc = 0.0820  # Constante solar (MJ·m⁻²·min⁻¹)
    phi = math.radians(lat_deg)  # Latitud en radianes
    dr = 1.0 + 0.033 * math.cos(2 * math.pi * J / 365.0)  # Distancia relativa Tierra-Sol
    delta = 0.409 * math.sin(2 * math.pi * J / 365.0 - 1.39)  # Declinación solar (rad)
    ws = math.acos(-math.tan(phi) * math.tan(delta))  # Ángulo de puesta del sol (rad)
    return (24 * 60 / math.pi) * Gsc * dr * (
        ws * math.sin(phi) * math.sin(delta) +
        math.cos(phi) * math.cos(delta) * math.sin(ws)
    )

def Ra_MJm2day_vectorized(lat_deg: float, J_series: pd.Series) -> pd.Series:
    return J_series.apply(lambda J: Ra_MJm2day_scalar(lat_deg, int(J)))

def Rso_clear_sky_scalar(Ra: float, z: float) -> float:
    """Radiación en cielo despejado (MJ/m²/día)"""
    return (0.75 + 2e-5 * z) * Ra

def Rso_clear_sky_vectorized(Ra: pd.Series, z: float) -> pd.Series:
    return (0.75 + 2e-5 * z) * Ra

def Rns_shortwave_scalar(Rs: float, albedo: float = 0.23) -> float:
    """Radiación neta de onda corta (MJ/m²/día). Albedo≈0.23 para cultivos"""
    return (1.0 - albedo) * Rs

def Rns_shortwave_vectorized(Rs: pd.Series, albedo: float = 0.23) -> pd.Series:
    return (1.0 - albedo) * Rs

def Rn_longwave_scalar(Tmin: float, Tmax: float, ea: float,
                       Rs: float, Rso: float) -> float:
    """Radiación neta de onda larga (MJ/m²/día)"""
    sigma = 4.903e-9  # Constante de Stefan-Boltzmann (MJ·K⁻⁴·m⁻²·día⁻¹)
    Tkmax, Tkmin = Tmax + 273.16, Tmin + 273.16  # Conversión a Kelvin
    termT = (Tkmax ** 4 + Tkmin ** 4) / 2.0  # Término de temperatura
    termCld = 1.35 * (min(Rs / Rso, 1.0) if Rso > 1e-6 else 0.0) - 0.35  # Factor de nubosidad
    return sigma * termT * (0.34 - 0.14 * math.sqrt(max(ea, 0.01))) * termCld

def Rn_longwave_vectorized(Tmin: pd.Series, Tmax: pd.Series, ea: pd.Series,
                           Rs: pd.Series, Rso: pd.Series) -> pd.Series:
    sigma = 4.903e-9
    Tkmax, Tkmin = Tmax + 273.16, Tmin + 273.16
    termT = (Tkmax ** 4 + Tkmin ** 4) / 2.0
    termCld = 1.35 * (Rs / Rso.clip(lower=1e-6)).clip(upper=1.0) - 0.35
    return sigma * termT * (0.34 - 0.14 * np.sqrt(ea.clip(lower=0.01))) * termCld

def Rn_net_scalar(Tmin: float, Tmax: float, ea: float,
                  Ra: float, z: float, Rs: float,
                  albedo: float = 0.23) -> float:
    """Radiación neta total (MJ/m²/día)"""
    if Rs is None:
        raise ValueError("Falta Rs para estimar la radiación neta.")
    Rso = Rso_clear_sky_scalar(Ra, z)  # Radiación en cielo despejado
    Rns = Rns_shortwave_scalar(Rs, albedo)  # Radiación neta de onda corta
    RnL = Rn_longwave_scalar(Tmin, Tmax, ea, Rs, Rso)  # Radiación neta de onda larga
    return Rns - RnL

def Rn_net_vectorized(Tmin: pd.Series, Tmax: pd.Series, ea: pd.Series,
                      Ra: pd.Series, z: float, Rs: pd.Series,
                      albedo: float = 0.23) -> pd.Series:
    if Rs is None:
        raise ValueError("Falta Rs para estimar la radiación neta.")
    Rso = Rso_clear_sky_vectorized(Ra, z)
    Rns = Rns_shortwave_vectorized(Rs, albedo)
    RnL = Rn_longwave_vectorized(Tmin, Tmax, ea, Rs, Rso)
    return Rns - RnL

def wind_to_2m_scalar(Uz: float, z: float) -> float:
    """Corrección de viento medido a altura z para altura estándar de 2 m"""
    return Uz * (4.87 / math.log(67.8 * z - 5.42))

def wind_to_2m_vectorized(Uz: pd.Series, z: float) -> pd.Series:
    return Uz * (4.87 / np.log(67.8 * z - 5.42))

def eto_fao56_scalar(Tmin: float, Tmax: float, RHmin: float,
                     RHmax: float, Uz: float, z_viento: float,
                     z_terreno: float, lat_deg: float, J: int,
                     Rs: float) -> float:
    """
    Evapotranspiración de referencia según método FAO-56 Penman-Monteith (mm/día).
    Implementa íntegramente la ecuación (3) del marco teórico.
    """
    Tmean = 0.5 * (Tmin + Tmax)  # Temperatura media diaria (°C)
    es = es_daily_scalar(Tmin, Tmax)  # Presión de vapor de saturación (kPa)
    ea = min(ea_from_RH_scalar(Tmin, Tmax, RHmin, RHmax), es)  # Presión real de vapor (kPa)
    Delta = delta_slope_scalar(Tmean)  # Pendiente de la curva de presión de vapor (kPa/°C)
    lam = lambda_from_T_scalar(Tmean)  # Calor latente de vaporización (MJ/kg)
    P = pressure_from_z_scalar(z_terreno)  # Presión atmosférica (kPa)
    gamma = psychrometric_gamma_scalar(P, lam)  # Constante psicrométrica (kPa/°C)
    U2 = wind_to_2m_scalar(Uz, z_viento)  # Velocidad del viento a 2m (m/s)
    Ra = Ra_MJm2day_scalar(lat_deg, J)  # Radiación extraterrestre (MJ/m²/día)
    Rn = Rn_net_scalar(Tmin, Tmax, ea, Ra, z_terreno, Rs=Rs)  # Radiación neta (MJ/m²/día)
    G = 0.0  # Flujo de calor del suelo ≈0 a escala diaria
    
    # Ecuación FAO-56 Penman-Monteith
    numerator = (0.408 * Delta * (Rn - G) +
                 gamma * (900.0 / (Tmean + 273.0)) * U2 * (es - ea))
    denominator = Delta + gamma * (1.0 + 0.34 * U2)
    return numerator / denominator

def eto_fao56_vectorized(Tmin: pd.Series, Tmax: pd.Series, RHmin: pd.Series,
                         RHmax: pd.Series, Uz: pd.Series, z_viento: float,
                         z_terreno: float, lat_deg: float, J_series: pd.Series,
                         Rs: pd.Series) -> pd.Series:
    """Versión vectorizada de ETo para series temporales"""
    Tmean = 0.5 * (Tmin + Tmax)
    es = es_daily_vectorized(Tmin, Tmax)
    ea = np.minimum(ea_from_RH_vectorized(Tmin, Tmax, RHmin, RHmax), es)
    Delta = delta_slope_vectorized(Tmean)
    lam = lambda_from_T_vectorized(Tmean)
    size = len(Tmin)
    P = pressure_from_z_vectorized(z_terreno, size)
    gamma = psychrometric_gamma_vectorized(P, lam)
    U2 = wind_to_2m_vectorized(Uz, z_viento)
    Ra = Ra_MJm2day_vectorized(lat_deg, J_series)
    Rn = Rn_net_vectorized(Tmin, Tmax, ea, Ra, z_terreno, Rs=Rs)
    G = pd.Series(0.0, index=Tmin.index)
    
    numerator = (0.408 * Delta * (Rn - G) +
                 gamma * (900.0 / (Tmean + 273.0)) * U2 * (es - ea))
    denominator = Delta + gamma * (1.0 + 0.34 * U2)
    return numerator / denominator

# ---------------------------------------------------------------------
# Enfoque Dual Kc (FAO-56)
# ---------------------------------------------------------------------

def kc_max(U2: float, Rhmin: float, h_m: float, Kcb: float) -> float:
    """
    Coeficiente de cultivo máximo tras evento de mojado. Ajusta por viento, humedad
    y altura del dosel. Limitado entre 1.05 y 1.3, y nunca menor que Kcb + 0.05
    para mantener coherencia física.
    """
    base = 1.2 + (0.04 * (U2 - 2.0) - 0.004 * (Rhmin - 45.0)) * (h_m / 3.0) ** 0.3
    return max(1.05, min(1.3, max(base, Kcb + 0.05)))

def kc_max_vectorized(U2: pd.Series, Rhmin: pd.Series, h_m: pd.Series, Kcb: pd.Series) -> pd.Series:
    base = 1.2 + (0.04 * (U2 - 2.0) - 0.004 * (Rhmin - 45.0)) * (h_m / 3.0) ** 0.3
    return np.maximum(1.05, np.minimum(1.3, np.maximum(base, Kcb + 0.05)))

def few_from_fc_fw(fc: float, fw: float) -> float:
    """
    Fracción de suelo húmedo expuesto. few = min(1-fc, fw), donde fc es fracción
    cubierta por vegetación y fw es fracción mojada por riego/lluvia. Determina
    la superficie disponible para evaporación directa.
    """
    fc = max(0.0, min(1.0, fc))
    fw = max(0.0, min(1.0, fw))
    return max(0.0, min(1.0 - fc, fw))

def few_from_fc_fw_vectorized(fc: pd.Series, fw: pd.Series) -> pd.Series:
    fc = fc.clip(0, 1)
    fw = fw.clip(0, 1)
    return np.maximum(0.0, np.minimum(1.0 - fc, fw))

def kr_from_de(De: float, TEW: float, REW: float) -> float:
    """
    Coeficiente de reducción de evaporación del suelo. Kr=1 cuando De≤REW
    (fase 1: evaporación limitada por energía), y disminuye linealmente cuando
    De>REW (fase 2: evaporación limitada por disponibilidad de agua superficial).
    """
    if De <= REW:
        return 1.0
    if TEW <= REW:
        return 0.0
    return max(0.0, min(1.0, (TEW - De) / (TEW - REW)))

def kr_from_de_vectorized(De: pd.Series, TEW: pd.Series, REW: pd.Series) -> pd.Series:
    kr = pd.Series(1.0, index=De.index)
    mask = De > REW
    kr[mask] = (TEW[mask] - De[mask]) / (TEW[mask] - REW[mask])
    kr[kr < 0] = 0.0
    return kr

def kcb_linear_from_fc(fc: float, kcb_init: float, kcb_mid: float) -> float:
    """
    Interpolación lineal de Kcb según fracción de cobertura. Permite transición
    suave entre fase inicial (kcb_init) y desarrollo pleno (kcb_mid) del cultivo.
    """
    fc = max(0.0, min(1.0, fc))
    return kcb_init + (kcb_mid - kcb_init) * fc

def kcb_linear_from_fc_vectorized(fc: pd.Series, kcb_init: pd.Series, kcb_mid: pd.Series) -> pd.Series:
    fc = fc.clip(0, 1)
    return kcb_init + (kcb_mid - kcb_init) * fc

def kcb_adjust_climate_height(Kcb: float, U2: float, Rhmin: float, h_m: float) -> float:
    """
    Ajuste climático de Kcb según FAO-56. Corrige por condiciones locales de viento,
    humedad mínima y altura del cultivo. Esencial para adaptar valores tabulados a
    condiciones específicas de parcela.
    """
    adjustment = (0.04 * (U2 - 2.0) - 0.004 * (Rhmin - 45.0)) * (h_m / 3.0) ** 0.3
    return max(0.1, min(1.3, Kcb + adjustment))

def kcb_adjust_climate_height_vectorized(Kcb: pd.Series, U2: pd.Series, 
                                        Rhmin: pd.Series, h_m: pd.Series) -> pd.Series:
    adjustment = (0.04 * (U2 - 2.0) - 0.004 * (Rhmin - 45.0)) * (h_m / 3.0) ** 0.3
    return np.maximum(0.1, np.minimum(1.3, Kcb + adjustment))

class DualKcStrategy:
    """
    Implementación del enfoque dual Kc = Kcb·Ks + Ke (FAO-56, ecuación 11).
    Separa el componente basal de transpiración (Kcb) del componente de evaporación
    superficial (Ke), permitiendo que solo Kcb se vea afectado por estrés hídrico (Ks).
    """
    def __call__(self, x: PasoEntrada, ETo: NumberOrSeries, Ks: NumberOrSeries) -> Dict[str, NumberOrSeries]:
        is_series = isinstance(ETo, pd.Series)
        
        def to_series(v, default_idx=None):
            if isinstance(v, pd.Series):
                return v
            elif is_series and default_idx is not None:
                return pd.Series([v] * len(default_idx), index=default_idx)
            return v
        
        if is_series:
            idx = ETo.index
            Tmin = to_series(x.Tmin, idx)
            Tmax = to_series(x.Tmax, idx)
            Rhmin = to_series(x.RHmin, idx)
            Uz = to_series(x.Uz, idx)
            h_m = to_series(x.h_m, idx)
            fc = to_series(x.fc, idx)
            fw = to_series(x.fw, idx)
            lluvia = to_series(x.lluvia, idx)
            irrig = to_series(x.irrig, idx)
            kcb_init = to_series(x.kcb_init, idx)
            kcb_mid = to_series(x.kcb_mid, idx)
            De = to_series(x.De, idx)
            TEW = to_series(x.TEW, idx)
            REW = to_series(x.REW, idx)
            
            U2 = wind_to_2m_vectorized(Uz, x.z_viento)
            
            # Kcb: interpolación lineal o valor dado, seguido de ajuste climático
            Kcb_base = kcb_linear_from_fc_vectorized(fc, kcb_init, kcb_mid) if x.kcb_method == "linear" else kcb_mid
            Kcb = kcb_adjust_climate_height_vectorized(Kcb_base, U2, Rhmin, h_m)
            Kcmax = kc_max_vectorized(U2, Rhmin, h_m, Kcb)
            
            # Actualizar De con aportes del día antes de calcular Kr
            added = lluvia + irrig
            De_i = np.maximum(0.0, De - added)
            De_i = np.minimum(De_i, TEW)
            Kr = kr_from_de_vectorized(De_i, TEW, REW)
            
            # Ke: evaporación del suelo limitada por fracción expuesta y Kr
            few = few_from_fc_fw_vectorized(fc, fw)
            Ke_star = Kr * (Kcmax - Kcb)
            Ke_limit = few * Kcmax
            Ke = np.maximum(0.0, np.minimum(Ke_star, Ke_limit))
            
            # Kc total: solo Kcb se ve afectado por Ks (estrés hídrico)
            if Ks is not None:
                Kc = (Kcb * Ks) + Ke
            else:
                Kc = Kcb + Ke
            
            ETc = Kc * ETo
            Es = Ke * ETo
            De_end = np.minimum(TEW, De_i + Es)
            
            return {
                'ETc': ETc, 'Kcb': Kcb, 'Ke': Ke, 'Kc': Kc,
                'Kcmax': Kcmax, 'Kr': Kr, 'few': few, 'De_end': De_end, 'ETo': ETo
            }
        
        # Versión escalar
        U2 = wind_to_2m_scalar(float(x.Uz), x.z_viento)
        RHmin_val = float(x.RHmin)
        h_val = float(x.h_m)
        fc_val = float(x.fc)
        fw_val = float(x.fw)
        lluvia_val = float(x.lluvia)
        irrig_val = float(x.irrig)
        kcb_init_val = float(x.kcb_init)
        kcb_mid_val = float(x.kcb_mid)
        De_val = float(x.De)
        TEW_val = float(x.TEW)
        REW_val = float(x.REW)
        
        Kcb_base = kcb_linear_from_fc(fc_val, kcb_init_val, kcb_mid_val) if x.kcb_method == "linear" else kcb_mid_val
        Kcb = kcb_adjust_climate_height(Kcb_base, U2, RHmin_val, h_val)
        Kcmax = kc_max(U2, RHmin_val, h_val, Kcb)
        
        added = max(0.0, lluvia_val) + max(0.0, irrig_val)
        De_i = max(0.0, De_val - added)
        De_i = min(TEW_val, De_i)
        Kr = kr_from_de(De_i, TEW_val, REW_val)
        
        few = few_from_fc_fw(fc_val, fw_val)
        Ke_star = Kr * (Kcmax - Kcb)
        Ke_limit = few * Kcmax
        Ke = max(0.0, min(Ke_star, Ke_limit))
        
        Kc = (Kcb * float(Ks)) + Ke if Ks is not None else (Kcb + Ke)
        ETo_val = float(ETo)
        ETc = Kc * ETo_val
        Es = Ke * ETo_val
        De_end = min(TEW_val, De_i + Es)
        
        return {
            'ETc': ETc, 'Kcb': Kcb, 'Ke': Ke, 'Kc': Kc,
            'Kcmax': Kcmax, 'Kr': Kr, 'few': few, 'De_end': De_end, 'ETo': ETo_val
        }


# ---------------------------------------------------------------------
# Balance hídrico multicapa con conservación de masa
# ---------------------------------------------------------------------

def actualizar_balance_hidrico(perfil_suelo: PerfilSuelo, 
                               ETo: NumberOrSeries,
                               Ke: NumberOrSeries,
                               Kcb: NumberOrSeries,
                               Ks: NumberOrSeries,
                               Pe: NumberOrSeries,
                               irrig: NumberOrSeries,
                               lluvia: NumberOrSeries,
                               De_prev: NumberOrSeries,
                               TEW: NumberOrSeries,
                               check_masa: bool = True,
                               logger: Optional[logging.Logger] = None) -> PerfilSuelo:
    """
    Actualiza contenido de agua (θ) por capa mediante algoritmo de dos pasadas:
    
    1ª PASADA: Procesa capas secuencialmente aplicando:
       - Evaporación (E): solo en capa superficial (0-30 cm)
       - Transpiración (T): distribuida por pesos radiculares
       - Entradas: Pe + irrig (percolan), Psurf (humedece superficie)
       - Psurf representa humedecimiento superficial por lluvias pequeñas
    
    2ª PASADA: Redistribuye déficit de T no satisfecho entre capas con agua
    disponible, simulando capacidad adaptativa del sistema radicular.
    
    NOTA: 'irrig' debe venir como riego neto (eficiencia ya aplicada).
    """
    # Convertir todas las entradas a float (día único)
    E = float(Ke) * float(ETo)
    T_total = float(Kcb) * float(Ks) * float(ETo)
    Pe_val = float(Pe)
    irrig_neto = float(irrig)
    lluvia_val = float(lluvia)
    De_prev_val = float(De_prev)
    TEW_val = float(TEW)
    
    # Psurf: humedecimiento superficial incluso con lluvias pequeñas que no generan Pe
    psurf = max(0.0, min(lluvia_val + irrig_neto, max(0.0, TEW_val - De_prev_val)))
    
    perfil_actualizado = perfil_suelo.copiar()
    tm = thickness_map_from_profundidades(perfil_actualizado.profundidades)
    etiquetas = list(tm.keys())
    espesores = [tm[d] for d in etiquetas]
    
    # Almacenamiento inicial para verificación de cierre de masa
    almacen_ini = sum(float(perfil_actualizado.theta[d]) * tm[d] * 10.0 for d in etiquetas)
    
    # Distribución de transpiración según pesos radiculares
    T_por_capa = {d: T_total * perfil_actualizado.pesos[d] for d in etiquetas}
    
    # Entradas que percolan al perfil (Pe + riego)
    percolacion = Pe_val + irrig_neto
    
    # PRIMERA PASADA: extracción y percolación descendente
    deficit_transpiracion = 0.0
    extr_T_aplicada: Dict[str, float] = {d: 0.0 for d in etiquetas}
    
    for i, d in enumerate(etiquetas):
        esp = espesores[i]
        FC = perfil_actualizado.FC[d]
        PWP = perfil_actualizado.PWP[d]
        
        # Agua disponible tras entradas (mm)
        theta_prev = float(perfil_actualizado.theta[d])
        agua_mm = theta_prev * esp * 10.0 + percolacion + (psurf if i == 0 else 0.0)
        
        # Capacidad de extracción (no bajar de PWP)
        cap_mm = max(0.0, agua_mm - PWP * esp * 10.0)
        
        # Evaporación: solo en capa superficial
        extr_E = min(E, cap_mm) if i == 0 else 0.0
        cap_mm -= extr_E
        
        # Transpiración limitada por disponibilidad
        extr_T = min(T_por_capa[d], cap_mm)
        extr_T_aplicada[d] = extr_T
        
        # Aplicar extracciones
        agua_mm = max(0.0, agua_mm - extr_E - extr_T)
        
        # Acumular déficit no extraído
        deficit_transpiracion += float(T_por_capa[d] - extr_T)
        
        # Nuevo θ provisional
        nuevo_theta = agua_mm / (esp * 10.0)
        
        # Percolación hacia capa inferior si excede FC
        if nuevo_theta > FC:
            percolacion = (nuevo_theta - FC) * esp * 10.0
            nuevo_theta = FC
        else:
            percolacion = 0.0
        
        # Límite inferior en PWP
        if nuevo_theta < PWP:
            nuevo_theta = PWP
        
        perfil_actualizado.theta[d] = nuevo_theta
    
    # SEGUNDA PASADA: redistribución de déficit de transpiración
    if deficit_transpiracion > 1e-6:
        disponible: Dict[str, float] = {}
        for i, d in enumerate(etiquetas):
            esp = espesores[i]
            agua_mm = float(perfil_actualizado.theta[d]) * esp * 10.0
            disp = max(0.0, agua_mm - perfil_actualizado.PWP[d] * esp * 10.0)
            disponible[d] = disp
        
        total_disp = sum(disponible.values())
        if total_disp > 0.0:
            for i, d in enumerate(etiquetas):
                esp = espesores[i]
                cuota = deficit_transpiracion * (disponible[d] / total_disp)
                agua_mm = float(perfil_actualizado.theta[d]) * esp * 10.0
                extrae = min(cuota, max(0.0, agua_mm - perfil_actualizado.PWP[d] * esp * 10.0))
                perfil_actualizado.theta[d] = (agua_mm - extrae) / (esp * 10.0)
    
    # Verificación de cierre de masa (control de calidad numérica)
    if check_masa:
        almacen_fin = sum(float(perfil_actualizado.theta[d]) * tm[d] * 10.0 for d in etiquetas)
        delta = almacen_fin - almacen_ini
        entradas = Pe_val + irrig_neto + psurf
        salidas = E + sum(extr_T_aplicada.values())
        residuo = entradas - salidas - delta
        if abs(residuo) > 1.0:  # Tolerancia: 1 mm
            (logger or logging.getLogger(__name__)).warning(
                f"[Cierre de masa] Residuo={residuo:.2f} mm "
                f"(Entradas={entradas:.2f}, Salidas={salidas:.2f}, ΔAlm={delta:.2f})"
            )
    
    return perfil_actualizado

# ---------------------------------------------------------------------
# Orquestador principal - Capa 0
# ---------------------------------------------------------------------

class PreprocessCapa0:
    """
    Capa 0: Cálculo de indicadores biofísicos fundamentales.
    
    Implementa ETo (FAO-56 Penman-Monteith), RSWC multicapa, ETc (Dual Kc),
    Pe (precipitación efectiva) y señales de estado hídrico del perfil.
    
    Diseño: Cálculo puro sin efectos laterales. La actualización de estado se
    delega a simular_multiples_dias() para mantener separación de responsabilidades.
    """
    
    _instance_count = 0
    
    def __init__(self, p_ref: float = 0.4,
                 etc_strategy: Optional[ETcStrategy] = None,
                 enable_cache: bool = True, log_level: int = logging.INFO,
                 auto_fill_nan: bool = True):
        """
        Parámetros:
            p_ref: factor de agotamiento de referencia (0.4 según FAO-56 para vid)
            etc_strategy: estrategia de cálculo de ETc (por defecto DualKc)
            enable_cache: habilitar cache MD5 para ETo escalar
            auto_fill_nan: rellenar automáticamente datos faltantes
        """
        PreprocessCapa0._instance_count += 1
        self.p_ref = p_ref
        self.etc = etc_strategy or DualKcStrategy()
        self.enable_cache = enable_cache
        self.auto_fill_nan = auto_fill_nan
        self.cache: Dict[str, NumberOrSeries] = {}
        self.logger = logging.getLogger(f"PreprocessCapa0_{self._instance_count}")
        self.logger.setLevel(log_level)
        
        self._last_qc_notes: List[str] = []
    
    def __validar_inputs(self, x: PasoEntrada) -> None:
        """
        Validación exhaustiva de entradas: rangos físicos, coherencia y completitud.
        Primer nivel de defensa para garantizar calidad de datos en capas superiores.
        """
        if x.kcb_method not in ["linear", "given"]:
            raise ValueError("kcb_method debe ser 'linear' o 'given'.")
        if x.perfil_suelo is None:
            raise ValueError("Se requiere perfil_suelo.")
        x.perfil_suelo.validar()
        
        # Validación de rangos físicamente plausibles (ver documentación capítulo 4.2)
        validar_rango(x.Tmin, "Tmin", -30, 40)
        validar_rango(x.Tmax, "Tmax", -10, 55)
        validar_rango(x.RHmin, "RHmin", 0, 100)
        validar_rango(x.RHmax, "RHmax", 0, 100)
        validar_rango(x.Uz, "Uz", 0, 50)
        validar_rango(x.J, "J", 1, 366)
        validar_rango(x.Rs, "Rs", 0, 50)
        validar_rango(x.h_m, "h_m", 0, 3)
        validar_rango(x.fc, "fc", 0, 1)
        validar_rango(x.fw, "fw", 0, 1)
        validar_rango(x.lluvia, "lluvia", 0, 100)
        validar_rango(x.lluvia_esperada_0_24h, "lluvia_esperada_0_24h", 0, 100)
        validar_rango(x.lluvia_esperada_24_48h, "lluvia_esperada_24_48h", 0, 100)
        validar_rango(x.irrig, "irrig", 0, 100)
        validar_rango(x.kcb_init, "kcb_init", 0, 1.5)
        validar_rango(x.kcb_mid, "kcb_mid", 0, 1.5)
        validar_rango(x.De, "De", 0, 100)
        validar_rango(x.TEW, "TEW", 0, 100)
        validar_rango(x.REW, "REW", 0, 100)
        
        self._last_qc_notes = []
        
        # Tratamiento de datos faltantes (series temporales)
        if isinstance(x.Tmin, pd.Series):
            for field in ['Tmin', 'Tmax', 'RHmin', 'RHmax', 'Uz']:
                series = getattr(x, field)
                if series.isna().any():
                    if self.auto_fill_nan:
                        if field in ['Tmin', 'Tmax']:
                            filled = series.interpolate()
                            self._last_qc_notes.append(f"{field}: interpolado")
                        else:
                            filled = series.ffill().bfill()
                            self._last_qc_notes.append(f"{field}: ffill/bfill")
                        object.__setattr__(x, field, filled)
                        self.logger.info(f"Relleno de NaN aplicado en {field}.")
            
            # Tratamiento especial para Rs (puede tener gaps)
            if isinstance(x.Rs, pd.Series) and x.Rs.isna().any() and self.auto_fill_nan:
                filled = x.Rs.interpolate(limit_direction='both')
                object.__setattr__(x, 'Rs', filled)
                self._last_qc_notes.append("Rs: interpolado")
                self.logger.info("Relleno de NaN aplicado en Rs (interpolación).")
            
            # Interpolación continua adicional para RH y Uz
            for field in ['RHmin', 'RHmax', 'Uz']:
                series = getattr(x, field)
                if isinstance(series, pd.Series):
                    if series.isna().any():
                        filled = series.interpolate(limit_direction='both')
                        object.__setattr__(x, field, filled)
                        self._last_qc_notes.append(f"{field}: interpolado (continua)")
                        self.logger.info(f"Interpolación continua aplicada en {field}.")
            
            # Chequeo de coherencia Tmin < Tmax
            if any(x.Tmin > x.Tmax):
                self.logger.warning("Se detectaron Tmin > Tmax en algunos registros.")
                self._last_qc_notes.append("Tmin>Tmax: warning")
            
            # Chequeo de plausibilidad Rs≈0 con viento alto (posible error de sensor)
            try:
                u2 = wind_to_2m_vectorized(x.Uz, x.z_viento)
                rs_low = (x.Rs < 0.5) if isinstance(x.Rs, pd.Series) else pd.Series(False, index=u2.index)
                wind_high = (u2 > 8.0)
                if (rs_low & wind_high).any():
                    self.logger.warning("Plausibilidad: Rs≈0 con u2>8 m/s detectado en algunas fechas.")
                    self._last_qc_notes.append("Plausibilidad: Rs≈0 con u2>8 m/s")
            except Exception:
                pass
        else:
            if float(x.Tmin) > float(x.Tmax):
                self.logger.warning("Tmin > Tmax en entrada escalar.")
                self._last_qc_notes.append("Tmin>Tmax: warning (escalar)")
    
    def __calc_ETo_optimized(self, x: PasoEntrada) -> NumberOrSeries:
        """
        Cálculo de ETo con optimización mediante cache MD5 para valores escalares.
        La arquitectura dual (escalar/vectorizado) garantiza eficiencia tanto para
        cálculos puntuales como para series temporales extensas.
        """
        cache_key = None
        use_cache = (not isinstance(x.Tmin, pd.Series)) and self.enable_cache
        
        if use_cache:
            cache_parts = [
                f"z_terreno:{x.z_terreno}", f"lat_deg:{x.lat_deg}", f"J:{x.J}",
                f"Tmin:{x.Tmin}", f"Tmax:{x.Tmax}", f"RHmin:{x.RHmin}",
                f"RHmax:{x.RHmax}", f"Uz:{x.Uz}", f"z_viento:{x.z_viento}", f"Rs:{x.Rs}"
            ]
            cache_key = hashlib.md5("|".join(map(str, cache_parts)).encode()).hexdigest()
            if cache_key in self.cache:
                return self.cache[cache_key]
        
        if isinstance(x.Tmin, pd.Series):
            idx = x.Tmin.index
            if isinstance(idx, pd.DatetimeIndex):
                J_series = idx.dayofyear
            else:
                J_series = x.J if isinstance(x.J, pd.Series) else pd.Series([x.J] * len(idx), index=idx)
            
            def get_series_or_default(v, default_val=0.0):
                if isinstance(v, pd.Series):
                    return v
                return pd.Series([default_val] * len(idx), index=idx)
            
            Rs_series = get_series_or_default(x.Rs) if x.Rs is not None else None

            result = eto_fao56_vectorized(
                x.Tmin, x.Tmax, x.RHmin, x.RHmax, x.Uz,
                x.z_viento, x.z_terreno, x.lat_deg, J_series,
                Rs_series
            )
        else:
            result = eto_fao56_scalar(
                float(x.Tmin), float(x.Tmax), float(x.RHmin), float(x.RHmax),
                float(x.Uz), x.z_viento, x.z_terreno, x.lat_deg, int(x.J),
                float(x.Rs) if x.Rs is not None else 0.0
            )
        
        if use_cache and cache_key is not None:
            self.cache[cache_key] = result
        return result
      
    def __calc_RSWC_optimized(self, x: PasoEntrada) -> NumberOrSeries:
        if x.perfil_suelo is None:
            raise ValueError("Se requiere perfil_suelo para RSWC.")
        return calc_RSWC_weighted(
            x.perfil_suelo.theta, x.perfil_suelo.FC, x.perfil_suelo.PWP, x.perfil_suelo.pesos
        )
    
    def __call__(self, x: PasoEntrada) -> PasoSalida:
        """
        Cálculo de indicadores biofísicos para un paso temporal.
        
        Secuencia: RSWC → ETo → TAW → p → RAW → Dr → Ks (aproximado) → ETc → Pe
        
        NOTA: Ks se aproxima considerando el efecto de las entradas del día sobre
        el perfil (mejora vs usar estado previo directo), aunque es una aproximación
        de 1 iteración. Para simulación multidía precisa usar simular_multiples_dias().
        """
        self.__validar_inputs(x)
        self.logger.info(f"Procesando datos para {len(x.Tmin) if hasattr(x.Tmin, '__len__') else 1} días")
        
        # 1) Contenido relativo de agua en el suelo
        RSWC = self.__calc_RSWC_optimized(x)
        
        # 2) Evapotranspiración de referencia
        ETo = self.__calc_ETo_optimized(x)
        
        # 3) Agua total disponible
        TAW = calc_TAW(x.perfil_suelo)
        
        # 4) Factor de agotamiento ajustado dinámicamente por ETo
        p = calc_p_adjusted(self.p_ref, ETo)
        
        # 5) Agua fácilmente utilizable
        RAW = calc_RAW(TAW, p)
        
        # Aproximación iterativa de Ks: aplicar entradas al perfil antes de calcular Dr
        # Esto mejora la estimación vs usar el estado previo directamente
        perfil_con_entradas = x.perfil_suelo.copiar()
        
        irrig_val = float(x.irrig) if not isinstance(x.irrig, pd.Series) else float(x.irrig.iloc[-1])
        lluvia_val = float(x.lluvia) if not isinstance(x.lluvia, pd.Series) else float(x.lluvia.iloc[-1])
        De_prev_val = float(x.De) if not isinstance(x.De, pd.Series) else float(x.De.iloc[-1])
        TEW_val = float(x.TEW) if not isinstance(x.TEW, pd.Series) else float(x.TEW.iloc[-1])
        
        # Calcular Pe y Psurf para aproximación
        Pe_val = max(0.0, 0.8 * (lluvia_val - 5.0))
        psurf = max(0.0, min(lluvia_val + irrig_val, max(0.0, TEW_val - De_prev_val)))
        
        # Aplicar entradas sin extracciones (aproximación conservadora)
        if Pe_val > 0 or irrig_val > 0 or psurf > 0:
            tm = thickness_map_from_profundidades(perfil_con_entradas.profundidades)
            etiquetas = list(tm.keys())
            percolacion = Pe_val + irrig_val
            
            for i, d in enumerate(etiquetas):
                esp = tm[d]
                theta_actual = float(perfil_con_entradas.theta[d])
                agua_mm = theta_actual * esp * 10.0 + percolacion + (psurf if i == 0 else 0.0)
                nuevo_theta = agua_mm / (esp * 10.0)
                
                if nuevo_theta > perfil_con_entradas.FC[d]:
                    percolacion = (nuevo_theta - perfil_con_entradas.FC[d]) * esp * 10.0
                    nuevo_theta = perfil_con_entradas.FC[d]
                else:
                    percolacion = 0.0
                
                perfil_con_entradas.theta[d] = nuevo_theta
        
        # 6) Déficit de agua con perfil actualizado
        Dr = calc_Dr(perfil_con_entradas)
        
        # 7) Coeficiente de estrés hídrico
        Ks = calc_Ks(Dr, TAW, RAW)
        
        # 8) Evapotranspiración del cultivo (Dual Kc)
        etc_pack = self.etc(x, ETo, Ks)
        ETc = etc_pack["ETc"]
        
        # 9) Precipitación efectiva actual
        Pe = calc_Pe(x.lluvia)
        
        # 10) Precipitación efectiva prevista (0-24h y 24-48h)
        pe_prevista_0_24h = calc_Pe(x.lluvia_esperada_0_24h)
        pe_prevista_24_48h = calc_Pe(x.lluvia_esperada_24_48h)
        
        # 11) Día juliano
        if isinstance(x.Tmin, pd.Series) and isinstance(x.Tmin.index, pd.DatetimeIndex):
            J = x.Tmin.index.dayofyear
        else:
            J = x.J
        
        # 12) Demanda neta diaria (para trazabilidad)
        DND = ETc - Pe
        
        return PasoSalida(
            Ks=Ks, J=J, pe_prevista_24_48h=pe_prevista_24_48h, pe_prevista_0_24h=pe_prevista_0_24h,
            Pe=Pe, ETc=ETc, RSWC=RSWC, DND=DND, TAW=TAW, RAW=RAW, Dr=Dr, p=p,
            Kcb=etc_pack.get("Kcb"), Ke=etc_pack.get("Ke"), Kc=etc_pack.get("Kc"),
            Kcmax=etc_pack.get("Kcmax"), Kr=etc_pack.get("Kr"), few=etc_pack.get("few"),
            De_end=etc_pack.get("De_end"), ETo=ETo,
            qc_notes=self._last_qc_notes.copy()
        )

# ---------------------------------------------------------------------
# Simulación multidía con actualización de estado
# ---------------------------------------------------------------------

def simular_multiples_dias(datos_diarios: List[PasoEntrada], 
                           estado_inicial: SimulationState,
                           p_ref: float = 0.4) -> Tuple[List[PasoSalida], List[PerfilSuelo]]:
    """
    Simula balance hídrico para múltiples días consecutivos.
    
    Encadena cálculos día a día actualizando θ por capa y De (agotamiento
    superficial) tras cada paso. Devuelve resultados y perfiles actualizados
    para trazabilidad completa del estado hídrico.
    
    Parámetros:
        datos_diarios: lista de PasoEntrada (un elemento por día)
        estado_inicial: estado inicial del perfil y De
        p_ref: factor de agotamiento de referencia (0.4 para vid)
    
    Retorna:
        (resultados, perfiles_actualizados): tupla con salidas y estados del perfil
    """
    processor = PreprocessCapa0(p_ref=p_ref)
    resultados: List[PasoSalida] = []
    perfiles_actualizados: List[PerfilSuelo] = []
    estado_actual = estado_inicial.copiar()
    
    for i, datos_dia in enumerate(datos_diarios):
        # Sincronizar perfil y De del estado acumulado
        if datos_dia.perfil_suelo is None:
            datos_dia = replace(datos_dia, perfil_suelo=estado_actual.perfil_suelo)
        datos_dia = replace(datos_dia, De=estado_actual.De)
        
        # 1) Calcular señales del día (cálculo puro, sin modificar estado)
        salida = processor(datos_dia)
        resultados.append(salida)
        
        # Extraer valores escalares para el balance (última posición si es serie)
        def last_or_float(v: NumberOrSeries) -> float:
            if isinstance(v, pd.Series):
                return float(v.iloc[-1])
            return float(v)
        
        ETo_val = last_or_float(salida.ETo)
        Ke_val  = last_or_float(salida.Ke)
        Kcb_val = last_or_float(salida.Kcb)
        Ks_val  = last_or_float(salida.Ks)
        Pe_val  = last_or_float(salida.Pe)
        irrig_val = last_or_float(datos_dia.irrig)
        lluvia_val = last_or_float(datos_dia.lluvia)
        De_prev_val = last_or_float(estado_actual.De)
        TEW_val = last_or_float(datos_dia.TEW)
        
        # 2) Actualizar perfil mediante balance hídrico (con Psurf y cierre de masa)
        perfil_act = actualizar_balance_hidrico(
            perfil_suelo=estado_actual.perfil_suelo,
            ETo=ETo_val,
            Ke=Ke_val,
            Kcb=Kcb_val,
            Ks=Ks_val,
            Pe=Pe_val,
            irrig=irrig_val,
            lluvia=lluvia_val,
            De_prev=De_prev_val,
            TEW=TEW_val,
            logger=logging.getLogger("capa0.simulacion")
        )
        perfiles_actualizados.append(perfil_act.copiar())
        
        # 3) Avanzar estado para el día siguiente
        estado_actual.perfil_suelo = perfil_act
        estado_actual.De = last_or_float(salida.De_end)
        estado_actual.day = i + 1
    
    return resultados, perfiles_actualizados


# EJEMPLO USO 

In [2]:

if __name__ == "__main__":
    logging.basicConfig(level=logging.INFO)

    # --- 1) Perfil de suelo ---
    perfil_suelo = PerfilSuelo(
        profundidades=['20', '40', '60', '90'],
        theta={'20': 0.24, '40': 0.22, '60': 0.20, '90': 0.19},
        FC={'20': 0.33, '40': 0.31, '60': 0.30, '90': 0.29},
        PWP={'20': 0.15, '40': 0.14, '60': 0.14, '90': 0.13},
        pesos={'20': 0.45, '40': 0.25, '60': 0.175, '90': 0.125}
    )
    perfil_suelo.validar()

    # --- 2) Entrada de un día (nombres adaptados al dataclass) ---
    x = PasoEntrada(
        Tmin=14.37, Tmax=24.10, RHmin=39.35, RHmax=100.0,
        Uz=1.07, z_viento=10.0, z_terreno=698.0, lat_deg=39.44, J=264,
        Rs=14.22,
        h_m=1.5, fc=0.35, fw=0.25,
        lluvia=0.0,
        lluvia_esperada_0_24h=0.0,
        lluvia_esperada_24_48h=15.0,
        irrig=0.0,
        kcb_init=0.15, kcb_mid=0.65, kcb_method="linear",
        De=6.0, TEW=15.0, REW=7.0,
        perfil_suelo=perfil_suelo
    )

    # --- 3) Cálculo directo con Capa 0 ---
    pre = PreprocessCapa0(p_ref=0.4)
    salida = pre(x)

    print("=== Cálculo directo ===")
    J_val = int(salida.J if not isinstance(salida.J, pd.Series) else salida.J.iloc[-1])
    getf = lambda v: float(v if not isinstance(v, pd.Series) else v.iloc[-1])
    print(f"Día juliano: {J_val}")
    print(f"Ks: {getf(salida.Ks):.3f}")
    print(f"ETo [mm/d]: {getf(salida.ETo):.2f}")
    print(f"ETc [mm/d]: {getf(salida.ETc):.2f}")
    print(f"Pe [mm]: {getf(salida.Pe):.2f}")
    print(f"Pe 0-24h [mm]: {getf(salida.pe_prevista_0_24h):.2f}")
    print(f"Pe 24-48h [mm]: {getf(salida.pe_prevista_24_48h):.2f}")
    print(f"RSWC [-]: {getf(salida.RSWC):.3f}")
    print(f"DND [mm]: {getf(salida.DND):.2f}")
    print("QC notes:", salida.qc_notes)

    # --- 4) Simulación de 1 día (actualiza θ y De con cierre de masa) ---
    estado0 = SimulationState(perfil_suelo=perfil_suelo, De=6.0, day=0)
    resultados, perfiles = simular_multiples_dias([x], estado0, p_ref=0.4)

    sal = resultados[0]
    perf_fin = perfiles[0]

    print("\n=== Simulación (estado actualizado) ===")
    print(f"Ks: {getf(sal.Ks):.3f}")
    print(f"ETc [mm/d]: {getf(sal.ETc):.2f}")
    print(f"De_end [mm]: {getf(sal.De_end):.2f}")
    print("θ final por capa:")
    for d in perf_fin.profundidades:
        print(f"  {d} cm -> θ={float(perf_fin.theta[d]):.3f}")

2025-10-06 12:14:00,829 - PreprocessCapa0_1 - INFO - Procesando datos para 1 días
2025-10-06 12:14:00,830 - PreprocessCapa0_2 - INFO - Procesando datos para 1 días


=== Cálculo directo ===
Día juliano: 264
Ks: 0.844
ETo [mm/d]: 2.63
ETc [mm/d]: 1.45
Pe [mm]: 0.00
Pe 0-24h [mm]: 0.00
Pe 24-48h [mm]: 9.00
RSWC [-]: 0.455
DND [mm]: 1.45
QC notes: []

=== Simulación (estado actualizado) ===
Ks: 0.844
ETc [mm/d]: 1.45
De_end [mm]: 6.78
θ final por capa:
  20 cm -> θ=0.235
  40 cm -> θ=0.219
  60 cm -> θ=0.199
  90 cm -> θ=0.190
